In [1]:
import os
import sys
import cv2
import mediapipe as mp
import pandas as pd
import pickle
import tqdm
from moviepy.editor import VideoFileClip
from moviepy.video.fx.mirror_x import mirror_x
from Modules import utils

videosTrain = "Videos/Train/Normal"
flippedVideosTrain = "Videos/Train/Flipped"

videosTest = "Videos/Test/Normal"
flippedVideosTest = "Videos/Test/Flipped"

mpHands = mp.solutions.hands.Hands()
mpFace = mp.solutions.face_mesh.FaceMesh()

I0000 00:00:1729701702.676223 35830851 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1729701702.687876 35830851 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro


In [2]:
def flipVideo(videoFile, videoPath, newVideoPath):
    videoPath = os.path.join(videoPath, videoFile)
    newVideoPath = os.path.join(newVideoPath, videoFile)

    if not os.path.exists(newVideoPath) and videoFile.endswith(".mp4"):
        sys.stdout = open(os.devnull, 'w')

        video = VideoFileClip(videoPath)
        flippedVideo = video.fx(mirror_x)
        flippedVideo.write_videofile(newVideoPath, codec="libx264")

        sys.stdout = sys.__stdout__
                
for videoFile in tqdm.tqdm(os.listdir(videosTrain)):
    flipVideo(videoFile, videosTrain, flippedVideosTrain)

for videoFile in tqdm.tqdm(os.listdir(videosTest)):
    flipVideo(videoFile, videosTest, flippedVideosTest)

W0000 00:00:1729701702.693373 35834928 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729701702.696359 35834928 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729701702.696369 35834917 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


W0000 00:00:1729701702.707714 35834917 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
100%|██████████| 97/97 [00:00<00:00, 47045.27it/s]


In [3]:
videoData = []
rawVideoData = []
frameSkip = 5

def processVideo(videoFile, videoPath):
    if not videoFile.endswith(".mp4"):
        return None, None
    
    thisVideo = {"SENTENCE": "", "COORDINATES": []}
    allVideoData = {"FILE": "", "HANDS": [], "FACE": []}

    sentence = videoFile.split('-')[0]

    videoPath = os.path.join(videoPath, videoFile)
    
    cap = cv2.VideoCapture(videoPath)
    
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    currentFrame = 0 

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        currentFrame += 1
        if currentFrame <= frameSkip or currentFrame > frameCount - frameSkip:
            continue
        
        coordinates = utils.processFrame(frame, mpHands, mpFace)
        hands, face = utils.getDataFromFrame(frame, mpHands, mpFace)

        allVideoData["HANDS"].append(hands)
        allVideoData["FACE"].append(face)
        
        thisVideo["COORDINATES"].append(coordinates)

    cap.release()
    cv2.destroyAllWindows()
    
    thisVideo["SENTENCE"] = sentence
    allVideoData["FILE"] = videoFile
    return thisVideo, allVideoData

In [4]:
for videoFile in tqdm.tqdm(os.listdir(videosTrain)):
    thisVideo, allVideoData = processVideo(videoFile, videosTrain)

    if thisVideo is None:
        continue
    else:
        videoData.append(thisVideo)
        rawVideoData.append(allVideoData)

100%|██████████| 385/385 [39:45<00:00,  6.20s/it]


In [5]:
for videoFile in tqdm.tqdm(os.listdir(flippedVideosTrain)):
    thisVideo, allVideoData = processVideo(videoFile, flippedVideosTrain)

    if thisVideo is None:
        continue
    else:
        thisVideo["SENTENCE"] = thisVideo["SENTENCE"] + "_INV_"
        videoData.append(thisVideo)
        rawVideoData.append(allVideoData)

100%|██████████| 385/385 [39:30<00:00,  6.16s/it]


In [6]:
df = pd.DataFrame(videoData)
unprocessedDF = pd.DataFrame(rawVideoData)

df["SENTENCE"] = df["SENTENCE"].str.replace("Baño", "Baño")
unprocessedDF["FILE"] = unprocessedDF["FILE"].str.replace("Baño", "Baño")

print("Unique classes: ", df["SENTENCE"].unique().shape)

Unique classes:  (64,)


In [7]:
with open ('Data/LENSEGUA.pkl', 'wb') as f:
    pickle.dump(df, f)

df.to_excel('Data/LENSEGUA.xlsx')

print("Data saved to 'Data/LENSEGUA.pkl'")
print("Shape of the data:", df.shape)
df.head()

Data saved to 'Data/LENSEGUA.pkl'
Shape of the data: (768, 2)


,SENTENCE,COORDINATES
0,Tu,"[[-23, 379, 4, 354, 33, 333, 64, 328, 87, 330,..."
1,Hoy,"[[], [-127, 566, -98, 535, -54, 526, -16, 536,..."
2,Que,"[[], [270, 714, 242, 726, 232, 751, 227, 776, ..."
3,Universidad,"[[], [-93, 484, -51, 483, -17, 503, 2, 525, 12..."
4,Como,"[[], [-117, 655, -84, 668, -65, 693, -52, 713,..."


In [8]:
with open ('Data/LENSEGUA_RAW.pkl', 'wb') as f:
    pickle.dump(unprocessedDF, f)

unprocessedDF.to_excel('Data/LENSEGUA_RAW.xlsx')
print("Unprocessed data saved to 'Data/LENSEGUA_RAW.pkl'")
print("Shape of the unprocessed data:", unprocessedDF.shape)
unprocessedDF.head()

Unprocessed data saved to 'Data/LENSEGUA_RAW.pkl'
Shape of the unprocessed data: (768, 3)


,FILE,HANDS,FACE
0,Tu-Der-3.mp4,[[landmark {\n x: 0.487129629\n y: 0.5540930...,"[None, None, None, None, None, None, None, Non..."
1,Hoy-Frente-4.mp4,[[landmark {\n x: 0.461057961\n y: 0.5892689...,[[landmark {\n x: 0.528064\n y: 0.180241063\...
2,Que-Izq-3.mp4,[[landmark {\n x: 0.663456559\n y: 0.7785301...,[[landmark {\n x: 0.533213139\n y: 0.2177261...
3,Universidad-Der-3.mp4,[[landmark {\n x: 0.515573382\n y: 0.6067475...,[[landmark {\n x: 0.551567614\n y: 0.2571026...
4,Como-Izq-3.mp4,[[landmark {\n x: 0.491495907\n y: 0.7319240...,[[landmark {\n x: 0.539612174\n y: 0.1967032...
